# Training example TPV 13-3D

TPV 13-3D is a benchmark exercise that is designed to test if computer codes that simulate dynamic earthquake rupture are working as intended [(Harris et al., SRL 2018)](https://pubs.geoscienceworld.org/ssa/srl/article/89/3/1146/530061/A-Suite-of-Exercises-for-Verifying-Dynamic). It was designed by the [SCEC/USGS Spontaneous Rupture Code Verification Project](https://strike.scec.org/cvws/) and features:

* spontaneous rupture on a 2D planar 60-degree dipping normal fault  
* homogeneous half-space: $V_p=5716\,m/s, V_s=3300\,m/s, \rho=2700\,kg/m^3$
* non-associative Drucker-Prager plasticity with yielding in shear 
  (Here we use reduced bulk friction and cohesion to highlight its effect)
* linear-slip weakening friction (LSW)
* initial stress conditions are depth-dependent and specified everywhere throughout the entire model volume
* prescribed nucleation zone, square, 3 x 3 km in size, locally lower static coefficient of friction
* strongly supershear rupture conditions

![](tpv13.jpg)

[Detailed benchmark description (SCEC)](https://strike.scec.org/cvws/tpv12_13docs.html)

## Mesh generation

We use Gmsh for mesh generation. The following steps are required:

* create the half-space domain (box)
* create a 60° dipping fault plane
* account for the nucleation patch (why? see [Pelties et al., 2014](https://gmd.copernicus.org/articles/7/847/2014/) and [Galis et al., 2015](https://academic.oup.com/gji/article/200/2/890/609776)) 
* set boundary conditions

Gmsh comes with its own scripting language with which one can describe the geometry passed to the automatic mesh generator.
We provide the script tpv13_training.geo and are going to generate a mesh in the next step.

In [ ]:
!gmsh -3 tpv13_training.geo

You should now have a file called "tpv13_training.msh".
In the next step, we translate this file into the efficient HDF5 format, which is going to be read in by SeisSol. HDF5 stands for the Hierarchical Data Format version 5, and is an open source file format that supports large, complex, heterogeneous data. HDF5 uses a "file directory" like structure that allows you to organize data within the file in many different structured ways, as you might do with files on your computer, and is built for fast I/O processing and storage.

We use our tool pumgen for mesh generation for SeisSol, which is [open-source software](  https://github.com/SeisSol/PUMGen), see also [SeisSol's documentation]( https://seissol.readthedocs.io/en/latest/meshing-with-pumgen.html).

In [ ]:
!pumgen -s msh2 tpv13_training.msh

The files "tpv13_training.puml.h5" and "tpv13_training.xdmf" were created.
While the .h5 file is read by SeisSol, the .xdmf file can be used to visualize the mesh, as in the following.

In [ ]:
import vtk
import pyvista as pv

reader = vtk.vtkXdmfReader()
reader.SetFileName('tpv13_training.xdmf')
reader.Update()
mesh = pv.wrap(reader.GetOutput())
pv.plot(mesh, cmap='BuGn', background='white', show_edges=True, jupyter_backend='panel')

Note that the mesh is refined close to the fault.

### Exercises
* Inspect the tpv13_training.geo file and try to comprehend the steps. Consult the [Gmsh documentation](https://gmsh.info/doc/texinfo/gmsh.html) about the BooleanFragments operation.
* Create a finer resolved mesh by adjusting the h_fault parameter. Do not change the .geo file but use the [*-setnumber*](https://gmsh.info/doc/texinfo/gmsh.html#Command_002dline-options) argument of the *gmsh* command. Try, for example, to mesh the nucleation zone with 300 m and the remaining fault with 600 m high-order element edge length.

## Run the code

In the following we start SeisSol using the provided parameters file.
You should adjust the OMP_NUM_THREADS variable to match the number of cores on your PC or laptop.
Note, that on larger clusters SeisSol uses an optimized hybrid MPI + OpenMP parallelization.

In [ ]:
!OMP_NUM_THREADS=4 SeisSol_Release_dhsw_4_elastic parameters.par

While SeisSol is running, you might want to checkout the [documentation of the input files](https://seissol.readthedocs.io/en/latest/parameter-file.html).

## Visualization

We now visualize the fault output generated by SeisSol.
Check out the [documentation](https://seissol.readthedocs.io/en/latest/fault-output.html#outputmask) for an explanation of the variable abbreviations.

In [ ]:
from ipywidgets import interact

reader = vtk.vtkXdmfReader()
reader.SetFileName('output/tpv13-fault.xdmf')
reader.Update()
cd = reader.GetOutput().GetCellData()
variables = [cd.GetArrayName(i) for i in range(cd.GetNumberOfArrays())]

@interact(t=(0.0, 8.0, 1.0), var=variables)
def plot(t=0.0, var='SRd'):
    reader.UpdateTimeStep(t)
    mesh = pv.wrap(reader.GetOutput())
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('white')
    plotter.add_mesh(mesh, cmap='Blues', scalars=var)
    plotter.view_xz()
    plotter.show(jupyter_backend='panel')

# Local-time stepping exercises

One of the key features of SeisSol is an efficient local-time stepping scheme, which clusters elements with similar time-steps. This method pays off for applications featuring big differences in element sizes, caused by geometric complexity ([Ulrich et al., 2021](https://eartharxiv.org/repository/view/39/)) and/or multi-physics such as fully coupled seismic-acoustic-tsunami simulations ([Krenz et al., 2021](https://arxiv.org/abs/2107.06640)) or poroelastic wave propagation ([Wolf et al., 2021](https://arxiv.org/abs/2108.10565)).  

A time-step cluster $c \in \mathbb{N}_0$ has the time-step
$$\Delta t_c = r^c \Delta t_{\text{min}},$$
where the rate $r \in \mathbb{N}$. In the parameter file the rate $r$ can be set with the *ClusteredLTS* parameter.

* Run SeisSol once with rate-2 local time-stepping (LTS) and once with global time-stepping (GTS).
* In the log, look for "Elapsed time (via clock_gettime)" for both LTS and GTS.
* Compute the speed-up and compare it to the theoretical speed-up due to LTS.

*Hint:* You may reduce the *EndTime* parameter for this exercise.

# Off-fault yielding and ground deformation exercise
Here we propose to compare the subsidence and uplift pattern of a model accouting for off-fault plasticity with those of a fully-elastic model.  
In that purpose:  
* Run a fully elastic model with a different output prefix. Change the prefix (OutputFile) in &Output namelist, and turn off plasticity (Plasticity = 0) in parameters.par.
* Add the new prefix to the prefixList of the code block below.
* Analyse differences.

In [ ]:
prefixList = ['tpv13']
nSim = len(prefixList)
# coordinates of the line end points
a = [0, -10e3, 0]
b = [0, 10e3, 0]
# create line polydata for vizualizing
line = pv.Line(a, b)
p = pv.Plotter(shape=(1, nSim), notebook=True)

def load_mesh(prefix):
    # Read xdmf data
    reader = vtk.vtkXdmfReader()
    reader.SetFileName(f'output/{prefix}-surface.xdmf')
    reader.Update()
    reader.UpdateTimeStep(5.)
    return pv.wrap(reader.GetOutput())

# Plot the line of the elevation profile over the surface displacement 
for i, pref in enumerate(prefixList):
    mesh = load_mesh(pref)

    p.subplot(0, i)
    p.set_background('lightgrey')
    p.add_mesh(mesh, scalars='W')
    p.add_mesh(line, color="white", line_width=10)
    p.add_point_labels(
        [a, b], ["A", "B"], font_size=20, point_color="red", text_color="red"
    )
    p.camera.zoom(3)
p.show(jupyter_backend='panel')
    
# Plot subsidence/uplift profile
for i, pref in enumerate(prefixList):
    load_mesh(pref).plot_over_line(
        a,
        b,
        resolution=10000,
        title=f"subsidence/uplift profile for {pref}",
        ylabel="vectical displacement (m)",
        scalars='W'
    )